In [2]:
%matplotlib inline

import numpy as np
import pandas as pd
import matplotlib
from statsmodels.tsa.arima_model import ARIMA

matplotlib.rcParams['figure.figsize'] = (16.0, 12.0)
matplotlib.style.use('ggplot')
pd.set_option('display.max_rows', 500)


In [3]:
# Read in average personal finance data
pf = pd.read_csv('average_personal_income_and_expense.csv')
